In [1]:
import os
import copy
import numpy as np

In [2]:
def aoc_2021_22_1(file_path):
    """--- Day 22: Reactor Reboot --- Part One"""
 
    with open(file_path) as f:
        aoc_read = f.read().split('\n')
    
    # read instruction list
    instruction_list = []
    for reboot in aoc_read:
        temp_on_off = reboot.split(' ')[0]
        if temp_on_off == 'on':
            temp_on_off = 1
        else:
            temp_on_off = 0
        temp_reboot = reboot.split(' ')[1].split(',')
        temp_reboot = [tuple([int(c) for c in r.split('=')[-1].split('..')]) for r in temp_reboot]
        instruction_list.append(tuple([temp_on_off] + temp_reboot))
    
    # -50 to 50
    cuboid_array = np.zeros((101, 101, 101))

    for instruction in instruction_list:
        temp_x_range = np.array(instruction[1]) + 50
        temp_y_range = np.array(instruction[2]) + 50
        temp_z_range = np.array(instruction[3]) + 50

        x_min = max(0, temp_x_range[0])
        x_max = min(101, temp_x_range[1] + 1)
        y_min = max(0, temp_y_range[0])
        y_max = min(101, temp_y_range[1] + 1)
        z_min = max(0, temp_z_range[0])
        z_max = min(101, temp_z_range[1] + 1)

        if x_min < x_max and y_min < y_max and z_min < z_max:
            cuboid_array[x_min:x_max, y_min:y_max, z_min:z_max] = instruction[0]

    no_cubes = len(np.where(cuboid_array == 1)[0])

    return no_cubes

In [3]:
aoc_2021_22_1('example1.txt')

590784

In [4]:
aoc_2021_22_1('input.txt')

611176

In [5]:
def aoc_2021_22_2(file_path):
    """--- Day 22: Reactor Reboot --- Part Two"""
    
    with open(file_path) as f:
        aoc_read = f.read().split('\n')

    # read instruction list
    instruction_list = []
    for reboot in aoc_read:
        temp_on_off = reboot.split(' ')[0]
        if temp_on_off == 'on':
            temp_on_off = 1
        else:
            temp_on_off = 0
        temp_reboot = reboot.split(' ')[1].split(',')
        temp_reboot = [tuple([int(c) for c in r.split('=')[-1].split('..')]) for r in temp_reboot]
        instruction_list.append(tuple([temp_on_off] + temp_reboot))

    # initialize all cube intersections
    len_instruction = len(instruction_list)
    cube_intersection_dict = {}
    current_intersection_list = [tuple([c]) for c in range(len_instruction)]
    intersection_minmax_dict = {}

    # calculate cube intersections
    while current_intersection_list:
        new_intersection_list = []

        # for all intersection cubes
        for cubes in current_intersection_list:
            x_min, x_max = instruction_list[cubes[-1]][1]
            y_min, y_max = instruction_list[cubes[-1]][2]
            z_min, z_max = instruction_list[cubes[-1]][3]

            try:
                intersection_x_min, intersection_y_min, intersection_z_min, intersection_x_max, intersection_y_max, intersection_z_max = intersection_minmax_dict[tuple(list(cubes))]

                x_min = max(x_min, intersection_x_min)
                y_min = max(y_min, intersection_y_min)
                z_min = max(z_min, intersection_z_min)

                x_max = min(x_max, intersection_x_max)
                y_max = min(y_max, intersection_y_max)
                z_max = min(z_max, intersection_z_max)
            except KeyError:
                pass

            # add another cube
            if x_min <= x_max and y_min <= y_max and z_min <= z_max:
                cube_intersection_dict[tuple(cubes)] = (x_max - x_min + 1) * (y_max - y_min + 1) * (z_max - z_min + 1)
                temp_new_intersection = [tuple(list(cubes) + [c]) for c in range(cubes[-1] + 1, len_instruction)]
                new_intersection_list += temp_new_intersection

                # prestore min max indices
                for temp_new in temp_new_intersection:
                    intersection_minmax_dict[temp_new] = (x_min, y_min, z_min, x_max, y_max, z_max)

        # update cube list
        current_intersection_list = copy.deepcopy(new_intersection_list)

    # first cube
    if not instruction_list[0][0]:
        raise ValueError
    on_cube_dict = {}
    on_cube_dict[(0, )] = 1

    # instructions to turn cubes on or off
    for i_instruction in range(1, len_instruction):
        on_off = instruction_list[i_instruction][0]

        for cube in copy.deepcopy(sorted(on_cube_dict)):
            cube_intersection = tuple(list(cube) + [i_instruction])

            # turn off intersection
            if cube_intersection in cube_intersection_dict:
                on_cube_dict[cube_intersection] = -on_cube_dict[cube]

        # turn on entire cube if instruction is turn on
        if on_off:
            on_cube_dict[(i_instruction, )] = 1

    no_cubes = sum([cube_intersection_dict[k] * v for k, v in on_cube_dict.items()])

    return no_cubes

In [6]:
aoc_2021_22_2('example2.txt')

2758514936282235

In [7]:
aoc_2021_22_2('input.txt')

1201259791805392